In [2]:
import warnings
warnings.filterwarnings('ignore')

### 1. 전처리된 파일 불러오기

In [3]:
import numpy as np
import pandas as pd

In [4]:
X_train = pd.read_csv('./X_train.csv') 
y_train = pd.read_csv('./y_train.csv')
X_test = pd.read_csv('./X_test.csv')

In [5]:
X_train.shape, y_train.shape, X_test.shape

((891, 22), (891, 1), (418, 22))

### 2. 앙상블 모델 사용하기
- 머신러닝에서 성능이 좋은 모델
- 여러 개의 DecisionTree를 사용하는 모델

#### 2.1 RandomForest 모델 사용

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
# 모델 생성
forest_model = RandomForestClassifier()

#### 2.2 교차검증

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(forest_model, X_train, y_train, cv = 5).mean()

0.8002196974452325

### 3. 모델 학습

In [9]:
forest_model.fit(X_train, y_train)

RandomForestClassifier()

### 4. 모델 평가

In [10]:
# 학습한 모델로 예측
pred = forest_model.predict(X_test)
pred
# 평가 결과 답안지 불러오기
sub = pd.read_csv('./data/titanic/gender_submission.csv')
# 평가 결과 답안지에 예측한 값 넣기
sub['Survived'] = pred
# 예측한 값 내보내기 ==> kaggle
sub.to_csv('kcy_submission03.csv', index = False)
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [12]:
# 하이퍼 파라미터 튜닝
# n_estimators를 50 ~ 250까지 바꿔가면서 결과 확인
for i in range(50, 81):
    forest_model1 = RandomForestClassifier(n_estimators = i)
    # 교차검증으로 평가
    result = cross_val_score(forest_model1, X_train, y_train, cv = 5).mean()
    print(f'n_estimators : {i}, score : {result}')

n_estimators : 50, score : 0.8002385286548239
n_estimators : 51, score : 0.8058439520431863
n_estimators : 52, score : 0.7968865733475614
n_estimators : 53, score : 0.7946456594061893
n_estimators : 54, score : 0.8058565061829139
n_estimators : 55, score : 0.8103508882053857
n_estimators : 56, score : 0.8047391877471595
n_estimators : 57, score : 0.8002573598644153
n_estimators : 58, score : 0.8092272926997678
n_estimators : 59, score : 0.8069487163392128
n_estimators : 60, score : 0.8125980792166217
n_estimators : 61, score : 0.799114933149206
n_estimators : 62, score : 0.8035967610319503
n_estimators : 63, score : 0.806973824618668
n_estimators : 64, score : 0.7991212102190697
n_estimators : 65, score : 0.8002134203753688
n_estimators : 66, score : 0.7957504237022158
n_estimators : 67, score : 0.8069549934090766
n_estimators : 68, score : 0.8069675475488042
n_estimators : 69, score : 0.80248571966606
n_estimators : 70, score : 0.7968614650681063
n_estimators : 71, score : 0.806967547

In [ ]:
# 1. n_estimators를 제한 --> for
#for i in range(50, 81): # 30 - 50 ~ 80, 1씩 증가하는 값
# 2. max_depth를 제한 --> for
#    for j in range(5, 10) : # 5 - 5, 6, 7, 8, 9
# 30 * 5 = 150
# 최적의 하이퍼파라미터 조합을 찾기 위해서는 여러 개의 for문이 필요 --> 복잡, 작업이 번거로워짐
# --> 이 문제를 한 번에 해결하는 방법
# 한 번에 여러 개의 하이퍼 파라미터를 튜닝해주는 함수 ==> GridSearchCV()

In [14]:
# 1. 하이퍼 파라미터 튜닝
# GridSearchCV(모델, 사용할 하이퍼 파라미터, cv)

# 1. 관심(궁금한)있는 하이퍼 파라미터 지정
# n_estimators = [50, 70, 90, 110, 130, 150] --> 50 ~ 150
# max_depth = [3, 4, 5, 6, 7, 8]
# max_features = [0.3, 0.4, 0.5, 0.6, 0.7]

# 2. 파라미터를 딕셔너리 형태로 저장 key : 사용할 하이퍼 파라미터, value : 범위설정
param = {'n_estimators' : [50, 70, 90, 110, 130, 150],
        'max_depth' : [3, 4, 5, 6, 7, 8],
        'max_features' : [0.3, 0.4, 0.5, 0.6, 0.7]}

### GridSearchCV() 불러오기

In [15]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(RandomForestClassifier(), param, cv = 5)

In [16]:
# 2. 학습
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8],
                         'max_features': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [50, 70, 90, 110, 130, 150]})

In [18]:
# 3. 최적의 파라미터 조합 찾기
grid.best_params_

{'max_depth': 7, 'max_features': 0.6, 'n_estimators': 130}

In [17]:
# 4. 최적의 파라미터 일 때 best_score
grid.best_score_

0.8339024543343166

In [19]:
# 자세하게 찾아보기
param = {'n_estimators' : [48, 49, 50, 51, 52],
        'max_depth' : [6, 7, 8, 9, 10],
        'max_features' : [0.6, 0.7, 0.8]}

In [20]:
grid1 = GridSearchCV(RandomForestClassifier(), param, cv = 5)
grid1.fit(X_train, y_train)
print(grid1.best_params_)
print(grid1.best_score_)

{'max_depth': 9, 'max_features': 0.6, 'n_estimators': 51}
0.8383842822170611


In [22]:
# {'max_depth': 9, 'max_features': 0.6, 'n_estimators': 51} ==> 베스트 모델 생성
forest_model1 = RandomForestClassifier(max_depth = 9, max_features = 0.6, n_estimators = 51)
forest_model1.fit(X_train, y_train)

RandomForestClassifier(max_depth=9, max_features=0.6, n_estimators=51)

In [ ]:
# 학습한 모델로 예측
pred = forest_model.predict(X_test)
pred
# 평가 결과 답안지 불러오기
# 평가 결과 답안지에 예측한 값 넣기
# 예측한 값 내보내기 => kaggle
sub = pd.read_csv('./data/titanic/gender_submission.csv')
sub['Survived'] = pred
sub.to_csv('kcy_submission04.csv', index = False)
sub